In [2]:
import pandas as pd
import numpy as np
import os

## 1、数据前处理

In [3]:
FilePath = os.path.abspath('.')
SuperPath = os.path.dirname(FilePath)
DocPath = os.path.join(SuperPath, '1_Data')
print(FilePath)
print(SuperPath)

d:\bian\Competition\Loan default forecast\2_model
d:\bian\Competition\Loan default forecast


In [4]:
test = pd.read_csv(os.path.join(DocPath, 'testA.csv'))
train = pd.read_csv(os.path.join(DocPath, 'train.csv'))

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 46 columns):
id                    200000 non-null int64
loanAmnt              200000 non-null float64
term                  200000 non-null int64
interestRate          200000 non-null float64
installment           200000 non-null float64
grade                 200000 non-null object
subGrade              200000 non-null object
employmentTitle       200000 non-null float64
employmentLength      188258 non-null object
homeOwnership         200000 non-null int64
annualIncome          200000 non-null float64
verificationStatus    200000 non-null int64
issueDate             200000 non-null object
purpose               200000 non-null int64
postCode              200000 non-null float64
regionCode            200000 non-null int64
dti                   199939 non-null float64
delinquency_2years    200000 non-null float64
ficoRangeLow          200000 non-null float64
ficoRangeHigh         200000 no

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 47 columns):
id                    800000 non-null int64
loanAmnt              800000 non-null float64
term                  800000 non-null int64
interestRate          800000 non-null float64
installment           800000 non-null float64
grade                 800000 non-null object
subGrade              800000 non-null object
employmentTitle       799999 non-null float64
employmentLength      753201 non-null object
homeOwnership         800000 non-null int64
annualIncome          800000 non-null float64
verificationStatus    800000 non-null int64
issueDate             800000 non-null object
isDefault             800000 non-null int64
purpose               800000 non-null int64
postCode              799999 non-null float64
regionCode            800000 non-null int64
dti                   799761 non-null float64
delinquency_2years    800000 non-null float64
ficoRangeLow          800000 non-

In [6]:
label = train[['id','isDefault']]

In [7]:
#查看样本是否平衡
label['isDefault'].value_counts()

0    640390
1    159610
Name: isDefault, dtype: int64

In [8]:
train = train.drop('isDefault', axis=1)

In [9]:
df = train.append(test)

In [10]:
df.shape

(1000000, 46)

In [11]:
df.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           1
employmentLength      58541
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
purpose                   0
postCode                  1
regionCode                0
dti                     300
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      521
revolBal                  0
revolUtil               658
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     1
policyCode                0
n0                    50381
n1                    50381
n2                    50381
n3                    50381
n4                  

In [12]:
df['n10'].unique()

array([ 7., 13., 11.,  9., 12., 19.,  8., 15., 10., 16.,  5.,  6., 23.,
       17., nan, 29., 21.,  4., 14., 54.,  3., 31., 26., 20., 18., 24.,
       42.,  2., 32., 28., 41., 37., 52., 25., 27., 39., 22., 43., 35.,
       33., 38., 34., 36., 40., 30., 45., 46.,  1., 44., 56., 51., 48.,
       49., 58.,  0., 47., 61., 55., 76., 53., 81., 50., 57., 66., 60.,
       72., 62., 77., 70., 82., 59., 80., 64., 63., 75., 74., 67., 68.,
       84., 90., 65.])

In [13]:
df = df.fillna(-1)

## 2、特征工程构建

In [14]:
df.columns

Index(['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
       'annualIncome', 'verificationStatus', 'issueDate', 'purpose',
       'postCode', 'regionCode', 'dti', 'delinquency_2years', 'ficoRangeLow',
       'ficoRangeHigh', 'openAcc', 'pubRec', 'pubRecBankruptcies', 'revolBal',
       'revolUtil', 'totalAcc', 'initialListStatus', 'applicationType',
       'earliesCreditLine', 'title', 'policyCode', 'n0', 'n1', 'n2', 'n3',
       'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14'],
      dtype='object')

In [ ]:
import re

In [16]:
#改字段格式
df['employmentLength'].str.split(' ')

0           [2, years]
1           [5, years]
2           [8, years]
3         [10+, years]
4                  NaN
5           [7, years]
6           [9, years]
7            [1, year]
8           [5, years]
9                  NaN
10        [10+, years]
11          [3, years]
12          [2, years]
13        [10+, years]
14          [2, years]
15          [2, years]
16          [9, years]
17        [<, 1, year]
18        [10+, years]
19          [9, years]
20          [4, years]
21        [<, 1, year]
22           [1, year]
23          [4, years]
24        [10+, years]
25           [1, year]
26          [5, years]
27          [7, years]
28        [10+, years]
29        [<, 1, year]
              ...     
199970      [7, years]
199971      [3, years]
199972    [10+, years]
199973      [5, years]
199974      [6, years]
199975      [3, years]
199976    [10+, years]
199977      [2, years]
199978       [1, year]
199979    [<, 1, year]
199980      [8, years]
199981      [3, years]
199982    [